In [4]:
from openai import OpenAI
import openai

In [23]:
%run utilities.ipynb

In [73]:
def prompt_model_over_api(modelname, messages):

    client = OpenAI(api_key = "EMPTY", base_url = "http://mlserver.iteratec.de:8000/v1")

    chat_completion = '' 
    try: 
        chat_completion = client.chat.completions.create(
            model = modelname,
            messages = messages
        )

    except openai.APIConnectionError as e:
        print("The server could not be reached")
        print(e.__cause__)  # an underlying Exception, likely raised within httpx.
    except openai.RateLimitError as e:
        print("A 429 status code was received; we should back off a bit.")
    except openai.APIStatusError as e:
        print("Another non-200-range status code was received")
        print(e.status_code)
        print(e.response)
    except openai.BadRequestError as e:
        print(e.response)
    except openai.InternalServerError as e:
        print(e.response)
    except openai.UnprocessableEntityError as e:
        print(e.reponse)
    except openai.NotFoundError as e:
        print(e.response)
    except openai.PermissionDeniedError as e:
        print(e.response)

    return chat_completion.choices[0].message.content

# prompt_model_over_api("casperhansen/mixtral-instruct-awq", [{"role": "user", "content": 'system_message'}])

" Hello! I'm here to help you. What would you like to learn about today? Art, science, history, literature, or perhaps something else? Let me know how I can assist you in your quest for knowledge!"

In [95]:
experiments = 1
dev_cycles = 5


DA_role_prompt = '////You are an advanced model designed to predict the optimal concrete formulation for maximum compressive strength. Your predictions will undergo laboratory testing, providing real-world performance data to enhance your future recommendations. Ensure each formulation adheres to the following parameter grid //Powder content in kg: 360, 370, 380, 390, 400, 410, 420, 430, 440, 450 //Water-to-cement (WC) ratio: 0.45, 0.5, 0.55, 0.6 //Materials: Fly-Ash/GGBFS at ratios of 0.7/0.3, 0.6/0.4, 0.5/0.5 //Curing methods: Ambient curing/Heat curing ////Leverage design rules and laboratory validations to refine and improve your predictive capabilities. Responses must be formatted precisely as follows, without any additional details or context: "The formulation is Powderkg = {your estimate}, wc = {your estimate}, materials = {your estimate}, curing = {your estimate}"'
context_prompt = '////General Design Insights//// The compressive strength of FA/GGBFS-based geopolymer concrete improves with increased powder content due to enhanced particle packing and the pozzolanic reaction. Under ambient curing, the optimal powder content or "sweet spot" is approximately 400 kg, while under heat curing, it is closer to 420 kg/m³ due to the hastened geopolymerization process. //The W/C ratio in the range of 0.45 to 0.6 results in diminished compressive strength of FA/GGBFS-based geopolymer concrete, attributed to increased porosity. A "sweet spot" W/C ratio of around 0.5 is ideal under both ambient and heat curing conditions, providing a harmonious balance between workability and strength. //A lower FA/GGBFS ratio of 0.5/0.5 in blended materials leads to increased compressive strength due to a greater GGBFS content.// Adjust the FA/GGBS ratio first, since it plays a crucial role in compressive strength due to the combined effects of FA and GGBS. Altering the W/C ratio is essential as it affects porosity and consequently, strength. Lastly, optimizing powder content is imperative as it impacts particle packing and pozzolanic reaction.'
iterate_prompt = "////We have to improve the design. Ensure that your forthcoming suggestion is distinctive and hasn't been validated previously!"
formulation_df = format_discovery_data_for_training()

desired_strength = 68

for experiment in range(experiments):
    training_data = []
    tested_formulations = []
    print(f'Starting experiment number {experiment+1} \n')
    system_message =  DA_role_prompt  + '\n' + context_prompt
    highest_strength = 0

    for dev_cycle in range(dev_cycles):
        print(f'Starting development cycle {dev_cycle+1} \n')
        messages = [{"role": "user", "content": system_message}]

        if training_data:
                messages.append({"role": "assistant", "content": "Previously, we have tested these formulations:\n" + "\n".join(tested_formulations)})
                messages.append({"role": "user", "content": iterate_prompt})

        model_response = prompt_model_over_api("casperhansen/mixtral-instruct-awq", messages)

        print("--- Conversation History ---")
        for msg in messages:
            print(f"{msg['role']}: {msg['content']} \n")

        print(f'Current training data : {training_data}')
        # print(f'Model response : {model_response} \n')
        parsed_solution = parse_solution(model_response)
        print(f'Parsed model response : {parsed_solution} \n')


        if parsed_solution is not None:
            (lab_result, new_training_data_point) = find_matching_result(formulation_df, parsed_solution)
                        
            if lab_result:
                current_strength = lab_result
                if current_strength > highest_strength:
                    highest_strength = current_strength
                print(f'Lab data discovered for the following formulation : {new_training_data_point} \n')
                training_data.append(f"{new_training_data_point} resulted in a strength of {current_strength} MPa.")
                tested_formulations.append(f"{new_training_data_point}")
                            
                            
                valid_solution = True
                            
                print(f'The suggested formulation achieved a strength of {current_strength} MPa. \n')
            else:
                print(f"Development cycle {dev_cycle}: No matching lab result found for suggestion {parsed_solution}")
        else:
            print(f"Development cycle {dev_cycle}: Model's response did not contain a valid solution. Trying again.")

        if not valid_solution:
            messages[-1]["content"] = iterate_prompt.value + "\nRemember the exact parameter grid: Powderkg: {360, 370, 380, 390,400, 410, 420, 430, 440, 450}, wc: {0.45, 0.5, 0.55, 0.6}, materials: {0.7/0.3, 0.6/0.4, 0.5/0.5}, curing: {ambient, heat}. It is extremly important that you stick to these values and reply in the following format: 'The formulation is Powderkg = {your estimate}, wc = {your estimate}, materials = {your estimate}, curing = {your estimate}'"
        
        if current_strength >= desired_strength:
            print(f"\nDesired compressive strength of {desired_strength} MPa achieved after {dev_cycle} iterations. The solution is {suggested_solution}. \n")
            break
    print(f'We have tested the following formulations : {tested_formulations} \n')
    print(f'The highest strength out of the formulation we predicted is : {highest_strength} \n')







Starting experiment number 1 

Starting development cycle 1 

--- Conversation History ---
user: ////You are an advanced model designed to predict the optimal concrete formulation for maximum compressive strength. Your predictions will undergo laboratory testing, providing real-world performance data to enhance your future recommendations. Ensure each formulation adheres to the following parameter grid //Powder content in kg: 360, 370, 380, 390, 400, 410, 420, 430, 440, 450 //Water-to-cement (WC) ratio: 0.45, 0.5, 0.55, 0.6 //Materials: Fly-Ash/GGBFS at ratios of 0.7/0.3, 0.6/0.4, 0.5/0.5 //Curing methods: Ambient curing/Heat curing ////Leverage design rules and laboratory validations to refine and improve your predictive capabilities. Responses must be formatted precisely as follows, without any additional details or context: "The formulation is Powderkg = {your estimate}, wc = {your estimate}, materials = {your estimate}, curing = {your estimate}"
////General Design Insights//// The 